In [1]:
import sys
sys.path.append('/home/jjwhit/rcGAN/')

In [2]:
import torch
import yaml
import types
import json

import numpy as np
import matplotlib.patches as patches

from data.lightning.MassMappingDataModule import MMDataModule
from utils.parse_args import create_arg_parser
from pytorch_lightning import seed_everything
from models.lightning.mmGAN import mmGAN
from utils.mri.math import tensor_to_complex_np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from scipy import ndimage

/home/jjwhit/rcGAN/fastmri/__init__.py:16: UserWarning: Could not retrieve fastmri version!
  warnings.warn("Could not retrieve fastmri version!")


In [4]:
# Import model here
test_plot_model = mmGAN.load_from_checkpoint('/share/gpu0/jjwhit/mass_map/mm_models/mmgan_training_3/checkpoint-epoch=88.ckpt')

In [ ]:
%run /home/jjwhit/rcGAN/scripts/mass_map/validate.py --config /home/jjwhit/rcGAN/configs/mass_map_test.yml --exp-name mmgan_training_3 --num-gpus 1

In [ ]:
%run /home/jjwhit/rcGAN/scripts/mass_map/test.py --exp-name mmgan_training_8 --num-figs 10

In [1]:
import torch
import os
import yaml
import types
import json
import numpy as np

import sys
dir = '/home/jjwhit/rcGAN/'
sys.path.append(dir)

from data.lightning.MassMappingDataModule import MMDataModule
from utils.parse_args import create_arg_parser
from models.lightning.mmGAN import mmGAN
from pytorch_lightning import seed_everything
from utils.embeddings import VGG16Embedding
from evaluation_scripts.mass_map_cfid.cfid_metric import CFIDMetric

/home/jjwhit/rcGAN/fastmri/__init__.py:16: UserWarning: Could not retrieve fastmri version!
  warnings.warn("Could not retrieve fastmri version!")


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
def load_object(dct):
    return types.SimpleNamespace(**dct)

In [4]:
torch.set_float32_matmul_precision('medium')
# seed_everything(1, workers=True)

with open(dir+'configs/mass_map.yml', 'r') as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)
    cfg = json.loads(json.dumps(cfg), object_hook=load_object)

In [5]:
dm = MMDataModule(cfg)
dm.setup()
val_loader = dm.val_dataloader()
best_epoch = -1
inception_embedding = VGG16Embedding()
best_cfid = 10000000
start_epoch = 80 #Will start saving models after 70 epochs
end_epoch = 100

/home/jjwhit/.conda/envs/cGAN/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jjwhit/.conda/envs/cGAN/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
)


In [ ]:
    with torch.no_grad():
        
        for epoch in range(start_epoch, end_epoch):
            print(f"VALIDATING EPOCH: {epoch}")
            try:
                model = mmGAN.load_from_checkpoint(checkpoint_path='/share/gpu0/jjwhit/mass_map/mm_models/mmgan_training_3' + f'/checkpoint-epoch={epoch}.ckpt')
            except Exception as e:
                print(e)
                continue

            if model.is_good_model == 0:
                print("NO GOOD: SKIPPING...")
                continue

            model = model.cuda()
            model.eval()

            cfid_metric = CFIDMetric(
                gan=model,
                loader=val_loader,
                image_embedding=inception_embedding,
                condition_embedding=inception_embedding,
                cuda=True,
                args=cfg,
                ref_loader=False,
                num_samps=1
            )

            cfids = cfid_metric.get_cfid_torch_pinv()

            cfid_val = np.mean(cfids)

            if cfid_val < best_cfid:
                best_epoch = epoch
                best_cfid = cfid_val

VALIDATING EPOCH: 80


Computing generated distribution:   2%|▏         | 1/55 [00:03<03:35,  3.99s/it]

torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])


Computing generated distribution:   5%|▌         | 3/55 [00:04<00:57,  1.10s/it]

torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])


Computing generated distribution:   7%|▋         | 4/55 [00:04<00:38,  1.32it/s]

torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])


Computing generated distribution:   9%|▉         | 5/55 [00:05<00:30,  1.66it/s]

torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])
torch.Size([1, 300, 300, 2])


Computing generated distribution:  11%|█         | 6/55 [00:05<00:22,  2.14it/s]